In [4]:
import os
from transformers import DistilBertTokenizer, DistilBertModel
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType
import torch
from sentence_transformers import SentenceTransformer

# Initialize DistilBERT tokenizer and model
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#model = DistilBertModel.from_pretrained('distilbert-base-uncased')
#model = SentenceTransformer('sentence-transformers/sentence-t5-base')
#model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

def get_embedding(chunk):
    """Get the embedding vector for a chunk of text"""
    return model.encode(chunk)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import json
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility, db

from pymilvus import connections
connections.connect(
  alias="default",
  host='localhost',
  port='19530'
)

In [6]:
from pymilvus import Collection
collection = Collection("python_docV2")      # Get an existing collection.
collection.load()


In [7]:
search_params = {
    "metric_type": "L2", 
}

In [16]:
#query_vector = get_embedding("how do i write a module in C or C++?")
query_vector = get_embedding("how do i use regular expressions in Pythnon?")
#time the search
import time
start = time.time()
res = collection.search(
    data=[query_vector],
    anns_field="embedding",
    param=search_params,
    limit=10,
    output_fields=["id", "filename", "sentence"]
)
end = time.time()
#print the time in ms
print("Time to search: ", (end - start)*1000, "ms")

print(res)

Time to search:  29.511451721191406 ms
['["id: 443851359161372071, distance: 0.6003910303115845, entity: {\'sentence\': \'using regular expressions now that weve looked at some simple regular expressions how do we actually use them in python\', \'id\': 443851359161372071, \'filename\': \'116.txt\'}", "id: 443851359161372090, distance: 0.690823495388031, entity: {\'sentence\': \'regular expressions will often be written in python code using this raw string notation\', \'id\': 443851359161372090, \'filename\': \'116.txt\'}", "id: 443851359161371971, distance: 0.8679239749908447, entity: {\'sentence\': \'regular expression howto author am kuchling amkamkca abstract this document is an introductory tutorial to using regular expressions in python with the re module\', \'id\': 443851359161371971, \'filename\': \'116.txt\'}", "id: 443851359161400823, distance: 0.9095194935798645, entity: {\'sentence\': \'mastering regular expressions\', \'id\': 443851359161400823, \'filename\': \'330.txt\'}",

In [21]:
['["id: 443851359161372071, distance: 0.6003910303115845, entity: {\'sentence\': \'using regular expressions now that weve looked at some simple regular expressions how do we actually use them in python\', \'id\': 443851359161372071, \'filename\': \'116.txt\'}", "id: 443851359161372090, distance: 0.690823495388031, entity: {\'sentence\': \'regular expressions will often be written in python code using this raw string notation\', \'id\': 443851359161372090, \'filename\': \'116.txt\'}", "id: 443851359161371971, distance: 0.8679239749908447, entity: {\'sentence\': \'regular expression howto author am kuchling amkamkca abstract this document is an introductory tutorial to using regular expressions in python with the re module\', \'id\': 443851359161371971, \'filename\': \'116.txt\'}", "id: 443851359161400823, distance: 0.9095194935798645, entity: {\'sentence\': \'mastering regular expressions\', \'id\': 443851359161400823, \'filename\': \'330.txt\'}", "id: 443851359161427309, distance: 0.93463134765625, entity: {\'sentence\': \'python 24s regular expression engine can match this pattern without problems\', \'id\': 443851359161427309, \'filename\': \'479.txt\'}", "id: 443851359161372404, distance: 0.9459158778190613, entity: {\'sentence\': \'unfortunately it exclusively concentrates on perl and javas flavours of regular expressions and doesnt contain any python material at all so it wont be useful as a reference for programming in python\', \'id\': 443851359161372404, \'filename\': \'116.txt\'}", "id: 443851359161371982, distance: 0.94928377866745, entity: {\'sentence\': \'in these cases you may be better off writing python code to do the processing while python code will be slower than an elaborate regular expression it will also probably be more understandable\', \'id\': 443851359161371982, \'filename\': \'116.txt\'}", "id: 443851359161400683, distance: 0.958281934261322, entity: {\'sentence\': \'pattern the regular expression pattern\', \'id\': 443851359161400683, \'filename\': \'330.txt\'}", "id: 443851359161372405, distance: 0.9785351157188416, entity: {\'sentence\': \'the first edition covered pythons nowremoved regex module which wont help you much\', \'id\': 443851359161372405, \'filename\': \'116.txt\'}", "id: 443851359161400326, distance: 0.9877127408981323, entity: {\'sentence\': \'regular expressions can contain both special and ordinary characters\', \'id\': 443851359161400326, \'filename\': \'330.txt\'}"]']

#for each in res:
#print the sentence

for each in res[0]:
    print(each.entity.sentence, each.entity.filename, each.entity.id)

using regular expressions now that weve looked at some simple regular expressions how do we actually use them in python 116.txt 443851359161372071
regular expressions will often be written in python code using this raw string notation 116.txt 443851359161372090
regular expression howto author am kuchling amkamkca abstract this document is an introductory tutorial to using regular expressions in python with the re module 116.txt 443851359161371971
mastering regular expressions 330.txt 443851359161400823
python 24s regular expression engine can match this pattern without problems 479.txt 443851359161427309
unfortunately it exclusively concentrates on perl and javas flavours of regular expressions and doesnt contain any python material at all so it wont be useful as a reference for programming in python 116.txt 443851359161372404
in these cases you may be better off writing python code to do the processing while python code will be slower than an elaborate regular expression it will also 

In [17]:
#append to a list the ids of the response considering the structure is["['id: 4917, distance: 0.6798818111419678, entity: {}', 'id: 10402, distance: 0.6981911659240723, entity: {}', 'id: 2335, distance: 0.707775354385376, entity: {}', 'id: 9312, distance: 0.7683041095733643, entity: {}', 'id: 9599, distance: 0.7878801822662354, entity: {}', 'id: 4622, distance: 0.8129417896270752, entity: {}', 'id: 2336, distance: 0.8555702567100525, entity: {}', 'id: 10408, distance: 0.8695672750473022, entity: {}', 'id: 11102, distance: 0.8719511032104492, entity: {}', 'id: 10665, distance: 0.8743786215782166, entity: {}']"]
#then we can use the ids to get the text from the files
ids = res[0].ids

print(ids)

[443851359161372071, 443851359161372090, 443851359161371971, 443851359161400823, 443851359161427309, 443851359161372404, 443851359161371982, 443851359161400683, 443851359161372405, 443851359161400326]


In [18]:
import json

def retrieve_sentences_from_ids(ids):
    # Load data from JSON file
    with open("sentence_data.json", 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    # Filter sentences with the given IDs
    retrieved_sentences = [record['sentence'] for record in data if record['id'] in ids]

    # Print retrieved sentences
    for idx, sentence in enumerate(retrieved_sentences, 1):
        print(f"{idx}. {sentence}")


retrieve_sentences_from_ids(ids)


FileNotFoundError: [Errno 2] No such file or directory: 'sentence_data.json'

In [3]:
print("Number of elements in collection: ", collection.num_entities)
print("Is empty: ", collection.is_empty)

Number of elements in collection:  0
Is empty:  True


In [ ]:
collection.release()